In [ ]:
import os
from langchain_core.globals import set_llm_cache
from langchain_aws import InMemorySemanticCache
from langchain_aws import ChatBedrock
from langchain_aws.embeddings import BedrockEmbeddings
import redis
from redis.cluster import RedisCluster as MemoryDB

## Initialize the ChatBedrock and embeddings 

In [ ]:
# Create the Anthropic Model
model_kwargs = {
    "temperature": 0,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\\n\\nHuman:"],
}

In [ ]:
# Use the Anthropic Claude model
llm = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0", model_kwargs=model_kwargs
)


In [ ]:
# Create a Titan Embeddings client
embeddings = BedrockEmbeddings()

## Connect to MemoryDB

In [ ]:
%%time
memorydb_host = os.environ.get("MEMORYDB_HOST", "localhost")
memorydb_port = os.environ.get("MEMORYDB_PORT", 6379)
# print(f"MemoryDB Url = {memorydb_host}:{memorydb_port}")
rc = MemoryDB(
    host=memorydb_host,
    port=memorydb_port,
    ssl=False,
    decode_responses=False,
    ssl_cert_reqs="none",
)
rc.ping()
# rc.flushall()

## Submit a query  without setting up cache

In [ ]:
%%time
response = llm.invoke("Tell me about mission to moon")
print(response.content)

## Enable MemoryDB for durable semantic caching 

In [ ]:
set_llm_cache(
    InMemorySemanticCache(
        redis_url=f"redis://{memorydb_host}:{memorydb_port}/ssl=True&ssl_cert_reqs=none",
        embedding=embeddings,
    )
)

### Submit a query to the LLM and Re-run the same block to see the improvement in response time. 

In [ ]:
%%time
response = llm.invoke("Tell me about mission to moon")
print(response.content)

In [ ]:
%%time
response = llm.invoke("Who first invented a telescope")
print(response.content)

In [ ]:
%%time
response = llm.invoke("Who first invented a car")
print(response.content)

In [ ]:
%%time
respone3 = llm.invoke("Who  first  a Telescope")
print(respone3.content)